In [54]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

We want to first confirm Benjamin's Model using LR on epc's of 38 or higher

In [55]:
df = pd.read_csv('High_polt_pow_ez.csv')
df = df.loc[df['epc'] >= 38]
df = df.loc[df['dist'] >= 0]
df = df.loc[df['dist'] <= 6]
df = df.loc[df['dist'] - round(df['dist']) == 0]
df.head()

,Unnamed: 0,epc,dist,pow
7589,7589,38.0,-0.0,0.231210
7594,7594,38.0,1.0,0.209023
7598,7598,38.0,2.0,0.185000
7603,7603,38.0,3.0,0.164510
7608,7608,38.0,4.0,0.143952


In [56]:
X = df[['dist']]
y = df['pow']

In [57]:
model = LinearRegression()
model.fit(X,y)
print('eq:', str(model.coef_[0])+'x', '+', model.intercept_)

eq: -0.01709460844280231x + 0.2188479071363546


these numbers are lame and not over the boardy so we're gonna gix that a bit

In [58]:
class Rounded_Reg:
    def __init__(self, coeffs, intercept, round_degree):
        self.coeffs = np.array([round(coeff, round_degree) for coeff in coeffs])
        self.intercept = round(intercept, round_degree)

    def predict(self, X):
        y_pred = []
        for i in range(len(X)):
            prediction = self.coeffs*np.array(X.iloc[i]) + self.intercept
            y_pred.append(prediction)
        return np.array(y_pred)

initialize the rounded model

In [59]:
rmodel = Rounded_Reg([-0.01709460844280231],  0.2188479071363546, 2)

In [60]:
print(rmodel.coeffs,rmodel.intercept)

[-0.02] 0.22


In [61]:
y_pred = rmodel.predict(X)
print(mean_absolute_error(y,y_pred))
print(r2_score(y,y_pred))

0.008043859840675476
0.89123151686623


Pretty much confirming Benjamin's Model :) We can check the other categories like this:

In [62]:
df1 = pd.read_csv('High_polt_pow_ez.csv')
df1 = df1.loc[df1['epc'] >= 38]
df1 = df1.loc[df1['dist'] >= 6]
df1 = df1.loc[df1['dist'] - round(df1['dist']) == 0]
df1 = df1.loc[df1['pow'] > .05]
df1.head()


,Unnamed: 0,epc,dist,pow
7616,7616,38.0,6.0,0.113786
7621,7621,38.0,7.0,0.099080
7626,7626,38.0,8.0,0.085887
7631,7631,38.0,9.0,0.075791
7636,7636,38.0,10.0,0.060619


In [63]:
X = df1[['dist']]
y = df1['pow']

In [64]:
model = LinearRegression()
model.fit(X,y)
print('eq:', str(model.coef_[0])+'x', '+', model.intercept_)

eq: -0.008940272761264485x + 0.16763202192257462


In [65]:
rmodel1 = Rounded_Reg([-0.008940272761264485], 0.16763202192257462, 2)
print(rmodel1.coeffs,rmodel1.intercept)

[-0.01] 0.17


In [66]:
y_pred = rmodel1.predict(X)
print(mean_absolute_error(y,y_pred))
print(r2_score(y,y_pred))

0.00993561034582357
0.7297604584296185


So rule of 17 looks pretty good. im gonna generalize now cause this is a little boring

In [67]:
def get_rmodel(lower_epc, upper_epc, lower_dist, upper_dist):
    df = pd.read_csv('High_polt_pow_ez.csv')
    df = df.loc[df['epc'] >= lower_epc]
    df = df.loc[df['epc'] <= upper_epc]
    df = df.loc[df['dist'] >= lower_dist]
    df = df.loc[df['dist'] <= upper_dist]
    df = df.loc[df['dist'] - round(df['dist']) == 0]
    df = df.loc[df['pow'] > .05]
    
    X = df[['dist']]
    y = df['pow']

    model = LinearRegression()
    model.fit(X,y)

    rmodel = Rounded_Reg(model.coef_, model.intercept_, 2)

    y_pred = rmodel.predict(X)
    error = mean_absolute_error(y, y_pred)
    r2 = r2_score(y,y_pred)

    return (rmodel.coeffs, rmodel.intercept, error, r2)

In [101]:
print(get_rmodel(30, 38, 0, 6))
print(get_rmodel(30, 38, 7, 12))

(array([-0.02]), 0.23, 0.007769757381679398, 0.9545376974741254)
(array([-0.01]), 0.16, 0.003208691977142857, 0.9182980859248118)


In [112]:
print(get_rmodel(25, 30, 0, 9))

(array([-0.02]), 0.22, 0.010629599852678572, 0.9457760267739276)


In [133]:
print(get_rmodel(22, 25, 0, 9))

(array([-0.02]), 0.23, 0.009903926508196721, 0.9612487920723322)


Checking a baby strat here:

In [129]:
    df = pd.read_csv('High_polt_pow_ez.csv')
    df = df.loc[df['epc'] >= 22]
    df = df.loc[df['dist'] >= 0]
    df = df.loc[df['dist'] - round(df['dist']) == 0]
    df = df.loc[df['pow'] > .05]

In [130]:
X = df[['dist']]
y = df['pow']

In [131]:
rmodel = Rounded_Reg([-.02], 22, 2)

In [132]:
y_pred = rmodel.predict(X)

mean_absolute_error(y,y_pred)

21.75495483503624

yikes...